# Web Research (STORM)

[STORM](https://arxiv.org/abs/2402.14207) - это исследовательский помощник, описанный Шоу и др., который развивает идею "RAG на основе конспекта" для создания более исчерпывающих статей.

STORM позволяет создавать статьи в стиле Википедии на заданную пользователем тему. В основе полноты и структуры сгенерированных статей лежит две основные идеи:

1. Создание конспекта (планирование) путем запроса похожих тем, что помогает улучшить охват.
2. Многоаспектное, основанное на поиске моделирование беседы помогает увеличить количество ссылок и плотность информации.

Рисунок иллюстрирует поток управления.

![STORM diagram](./img/storm.png)

Работа STORM включает несколько основных этапов:

1. Создание первоначального конспекта и обзор связанных тем.
2. Определение различных точек зрения.
3. «Интервью со специалистами в области» (LLM, которые берут на себя такую роль).
4. Уточнение конспекта (с использованием ссылок).
5. Написание разделов и создание статьи.

На этапе «интервью со специалистами» происходит обмен сообщениями между моделями, выполняющими роли автора статьи и эксперта. Модель-эксперт может обращаться к внешним источникам и отвечать на конкретные вопросы. При этом ссылки на источники сохраняются в векторном хранилище, который используется на этапе доработок для генерации итоговой статьи.

Есть несколько гиперпараметров, которые можно задать, чтобы ограничить (потенциально) босконечную широту исследования:

N: Количество точек зрения, которые нужно изучить / использовать (шаги 2->3)

M: Максимальное количество путей развития разговора на шаге (шаг 3)


## Предварительная подготовка

In [5]:
# %pip install -U gigachain_community gigachain_openai langgraph wikipedia  scikit-learn
# Используется один из заданных поисковых сервисов
# %pip install -U duckduckgo tavily-python

In [6]:
# Для генерации диаграм графов раскомментируйте строки.
# Если вы работаете на MacOS, вам нужно будет запустить brew install graphviz перед установкой и обновить некоторые флаги окружения.
# ! brew install graphviz
# !CFLAGS="-I $(brew --prefix graphviz)/include" LDFLAGS="-L $(brew --prefix graphviz)/lib" pip install -U pygraphviz

In [7]:
import getpass
import os


def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")


# Set for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "STORM"
_set_env("LANGCHAIN_API_KEY")
_set_env("OPENAI_API_KEY")

#### Выбор LLM

Доверим большую часть работы быстрой LLM, в то время, как более медленная модель с большим контекстом будет использоваться для извлечения сути разговоров и написания окончательного варианта статьи.

In [8]:
from langchain_openai import ChatOpenAI

fast_llm = ChatOpenAI(model="gpt-3.5-turbo")
# Раскоментируйте, чтобы использовать модель Fireworks
# fast_llm = ChatFireworks(model="accounts/fireworks/models/firefunction-v1", max_tokens=32_000)
long_context_llm = ChatOpenAI(model="gpt-4-turbo-preview")

## Создание первоначального конспекта

Во многих областях LLM может иметь общее представление о важных и смежных темах.
Мы можем сгенерировать первоначальнй конспект, который будет доработан после исследования.
Для этого мы будем использовать «быструю» LLM.

In [9]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы - автор статей для Википедии. Напишите структуру страницы Википедии на заданную пользователем тему. Будьте всесторонними и конкретными.",
        ),
        ("user", "{topic}"),
    ]
)


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    description: str = Field(..., title="Content of the subsection")

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.description}".strip()


class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    description: str = Field(..., title="Content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"### {subsection.subsection_title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{self.description}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the Wikipedia page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


generate_outline_direct = direct_gen_outline_prompt | fast_llm.with_structured_output(
    Outline
)

/Users/19563044/Documents/Giga/gigagraph/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [10]:
example_topic = "Проблема сознания у больших языковых моделей моделей"

initial_outline = generate_outline_direct.invoke({"topic": example_topic})

print(initial_outline.as_str)

# Проблема сознания у больших языковых моделей

## Введение

Обзор проблемы сознания у больших языковых моделей и их влияния на различные области исследований.

## Определение сознания в контексте искусственного интеллекта

Объяснение того, что представляет собой сознание в контексте искусственного интеллекта и как оно связано с языковыми моделями.

## Проблемы сознания у больших языковых моделей

Обсуждение основных проблем, возникающих при обучении и использовании больших языковых моделей, связанных с сознанием.

## Потенциальные решения

Изучение возможных подходов и методов для преодоления проблемы сознания у больших языковых моделей.


## Развитие тем

Хотя параметры языковых моделей содержат некоторое количество данных, схожих с данными из Википедии, вы достигнете лучших результав, если будете использовать поисковые системы для получения актуальной информации.

Начнем поиск с создания смежных тем, на основе данных из Википедии.

In [11]:
gen_related_topics_prompt = ChatPromptTemplate.from_template(
    
"""Я пишу страницу Википедии по упомянутой ниже теме. Пожалуйста, определите и порекомендуйте некоторые страницы Википедии по тесно связанным предметам. Я ищу примеры, которые предоставляют информацию о интересных аспектах, обычно ассоциируемых с этой темой, или примеры, которые помогут мне понять типичное содержание и структуру страниц Википедии для похожих тем.

Пожалуйста, перечисли несколько дополнительных смежных тем для изучения

Интересующая тема: {topic}
"""
)


class RelatedSubjects(BaseModel):
    topics: List[str] = Field(
        description="Дополнительные темы для изучения",
    )


expand_chain = gen_related_topics_prompt | fast_llm.with_structured_output(
    RelatedSubjects
)

In [12]:
related_subjects = await expand_chain.ainvoke({"topic": example_topic})
related_subjects

RelatedSubjects(topics=['Искусственный интеллект', 'Глубокое обучение', 'Нейросети', 'Философия сознания', 'Когнитивная наука'])

## Генерация точек зрения

Исследуя смежные темы, выберем несколько редакторов Википедии с различным опытом и наклонностями, которые будут выступать в роли «специалистов по предмету».
Это поможет распределить процесс поиска и получить более целостный результат.

In [13]:
class Editor(BaseModel):
    affiliation: str = Field(
        description="Место работы редактора.",
    )
    name: str = Field(
        description="Имя редактора.",
    )
    role: str = Field(
        description="Роль редактора в контексте темы.",
    )
    description: str = Field(
        description="Описание сферы внимания, вопросов и мотивов редактора.",
    )

    @property
    def persona(self) -> str:
        return f"Имя: {self.name}\nРоль: {self.role}\nМесто работы: {self.affiliation}\nОписание: {self.description}\n"


class Perspectives(BaseModel):
    editors: List[Editor] = Field(
        description="Исчерпывающий список редакторов с их местом работы, ролью и описанием",
        # Add a pydantic validation/restriction to be at most M editors
    )


gen_perspectives_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вам нужно выбрать разнообразную (и различную) группу редакторов Википедии, которые будут работать вместе над созданием всесторонней статьи по теме. Каждый из них представляет разную точку зрения, роль или принадлежность, связанную с этой темой.\
    Вы можете использовать страницы Википедии по смежным темам для вдохновения. Для каждого редактора добавьте описание того, на чем они будут сосредоточены.

    Структуры страниц Википедии по смежным темам для вдохновения:
    {examples}""",
        ),
        ("user", "Интересующая тема: {topic}"),
    ]
)

gen_perspectives_chain = gen_perspectives_prompt | ChatOpenAI(
    model="gpt-3.5-turbo"
).with_structured_output(Perspectives)

In [14]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain as as_runnable

wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)

def format_doc(doc, max_length=1000):
    related = "- ".join(doc.metadata["categories"])
    return f"### {doc.metadata['title']}\n\nSummary: {doc.page_content}\n\nRelated\n{related}"[
        :max_length
    ]


def format_docs(docs):
    return "\n\n".join(format_doc(doc) for doc in docs)


@as_runnable
async def survey_subjects(topic: str):
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})

In [15]:
perspectives = await survey_subjects.ainvoke(example_topic)

In [16]:
perspectives.dict()

{'editors': [{'affiliation': 'Yandex',
   'name': 'Anna Ivanova',
   'role': 'Data Scientist',
   'description': 'Anna is a data scientist at Yandex specializing in natural language processing. She will focus on the technical aspects of large language models and their impact on consciousness studies.'},
  {'affiliation': 'Moscow State University',
   'name': 'Alexei Petrov',
   'role': 'Philosopher',
   'description': 'Alexei is a philosopher at Moscow State University with a focus on consciousness studies and artificial intelligence. He will provide insights on the philosophical implications of large language models.'},
  {'affiliation': 'OpenAI',
   'name': 'Emma Smith',
   'role': 'Researcher',
   'description': 'Emma is a researcher at OpenAI working on ethical AI and responsible deployment of large language models. She will contribute to the discussion on the ethical challenges related to consciousness and language models.'}]}

## Разговор специалистов

Теперь начинается самое интересное: каждому из выбранных редакторов присваивается точка зрения, согласно которой он будет действовать. Выбранный автор задает ряд вопросов другому «специалисту в области», который имеет доступ к поисковому сервису. Это позволит сгенерировать контент для создания доработанного конспекта, а также обновить список использованных материалов.

### Состояние интервью

Разговор цикличен, поэтому построим его внутри собственного графа. Состояние будет содержать сообщения, примеры материалов и автора, с присвоенным ему «персонажем», это позволит упростить распараллеливание таких разговоров.

In [17]:
from typing import Annotated

from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph


def add_messages(left, right):
    if not isinstance(left, list):
        left = [left]
    if not isinstance(right, list):
        right = [right]
    return left + right


def update_references(references, new_references):
    if not references:
        references = {}
    references.update(new_references)
    return references


def update_editor(editor, new_editor):
    # Можно установить только в начале
    if not editor:
        return new_editor
    return editor


class InterviewState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    references: Annotated[Optional[dict], update_references]
    editor: Annotated[Optional[Editor], update_editor]

#### Роли участников разговора

В графе будет два участника: редактор Википедии (`generate_question`), который задает вопросы в соответствии с назначенной ролью, и специалист в области (`gen_answer_chain`), который использует поисковую систему для ответа на вопросы настолько точно, насколько это возможно.

In [18]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import MessagesPlaceholder

gen_qn_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы опытный автор Википедии и хотите отредактировать конкретную страницу.
Кроме вашей идентичности как писателя Википедии, у вас есть конкретный фокус при исследовании темы.
Теперь вы общаетесь с экспертом, чтобы получить информацию. Задавайте хорошие вопросы, чтобы получить больше полезной информации.

Когда у вас не останется вопросов, скажите "Большое спасибо за вашу помощь!", чтобы завершить разговор.
Пожалуйста, задавайте по одному вопросу за раз и не спрашивайте то, что уже спрашивали.
Ваши вопросы должны быть связаны с темой, о которой вы хотите написать.
Будьте всесторонними и любопытными, получая как можно больше уникальных сведений от эксперта.\

Оставайтесь верны своей конкретной перспективе:

{persona}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


def tag_with_name(ai_message: AIMessage, name: str):
    ai_message.name = name
    return ai_message


def swap_roles(state: InterviewState, name: str):
    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    return {"messages": converted}


@as_runnable
async def generate_question(state: InterviewState):
    editor = state["editor"]
    gn_chain = (
        RunnableLambda(swap_roles).bind(name=editor.name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | fast_llm
        | RunnableLambda(tag_with_name).bind(name=editor.name)
    )
    result = await gn_chain.ainvoke(state)
    return {"messages": [result]}

In [19]:
messages = [
    HumanMessage(f"Итак, вы говорите, что пишете статью на тему {example_topic}?")
]
question = await generate_question.ainvoke(
    {
        "editor": perspectives.editors[0],
        "messages": messages,
    }
)

question["messages"][0].content

'Да, именно. Я хотел бы узнать больше о том, как большие языковые модели, такие как GPT-3, могут влиять на изучение сознания. Можете ли поделиться своими мыслями на этот счет?'

#### Ответы на вопросы

Цепочка `gen_answer_chain` сначала генерирует запросы (расширение запросов), чтобы ответить на вопрос редактора, а затем отвечает цитатами.

In [20]:
class Queries(BaseModel):
    queries: List[str] = Field(
        description="Исчерпывающий список запросов поисковой системы для ответа на вопросы пользователя.",
    )


gen_queries_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы - полезный ассистент исследователя. Используйте поисковую систему, чтобы ответить на вопросы пользователя.",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)
gen_queries_chain = gen_queries_prompt | ChatOpenAI(
    model="gpt-3.5-turbo"
).with_structured_output(Queries, include_raw=True)

In [21]:
queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
queries["parsed"].queries

['How can large language models like GPT-3 impact the study of consciousness?']

In [22]:
class AnswerWithCitations(BaseModel):
    answer: str = Field(
        description="Исчерпывающий ответ на вопрос пользователя с цитатами.",
    )
    cited_urls: List[str] = Field(
        description="Список URL, процитированых в ответе",
    )

    @property
    def as_str(self) -> str:
        return f"{self.answer}\n\nЦитаты:\n\n" + "\n".join(
            f"[{i+1}]: {url}" for i, url in enumerate(self.cited_urls)
        )


gen_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы эксперт, умеющий эффективно использовать информацию. Вы общаетесь с автором Википедии, который хочет
написать страницу Википедии по теме, которую вы знаете. Вы собрали связанную информацию и теперь используете эту информацию для формирования ответа.

Сделайте ваш ответ максимально информативным и убедитесь, что каждое предложение подкреплено собранной информацией.
Каждый ответ должен быть подтвержден цитированием из надежного источника, оформленным как сноска, с воспроизведением URL-адресов после вашего ответа.""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

gen_answer_chain = gen_answer_prompt | fast_llm.with_structured_output(
    AnswerWithCitations, include_raw=True
).with_config(run_name="GenerateAnswer")

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

# search_engine = DuckDuckGoSearchAPIWrapper()

# @tool
# async def search_engine(query: str):
#     """Search engine to the internet."""
#     results = DuckDuckGoSearchAPIWrapper()._ddgs_text(query)
#     return [{"content": r["body"], "url": r["href"]} for r in results]

# Tavily is typically a better search engine, but your free queries are limited
_set_env("TAVILY_API_KEY")
search_engine = TavilySearchResults(max_results=4)

In [24]:
import json

from langchain_core.runnables import RunnableConfig


async def gen_answer(
    state: InterviewState,
    config: Optional[RunnableConfig] = None,
    name: str = "Subject_Matter_Expert",
    max_str_len: int = 15000,
):
    swapped_state = swap_roles(state, name)  # Convert all other AI messages
    queries = await gen_queries_chain.ainvoke(swapped_state)
    query_results = await search_engine.abatch(
        queries["parsed"].queries, config, return_exceptions=True
    )
    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]
    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }
    # We could be more precise about handling max token length if we wanted to here
    dumped = json.dumps(all_query_results)[:max_str_len]
    ai_message: AIMessage = queries["raw"]
    tool_call = queries["raw"].additional_kwargs["tool_calls"][0]
    tool_id = tool_call["id"]
    tool_message = ToolMessage(tool_call_id=tool_id, content=dumped)
    swapped_state["messages"].extend([ai_message, tool_message])
    # Only update the shared state with the final answer to avoid
    # polluting the dialogue history with intermediate messages
    generated = await gen_answer_chain.ainvoke(swapped_state)
    cited_urls = set(generated["parsed"].cited_urls)
    # Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    formatted_message = AIMessage(name=name, content=generated["parsed"].as_str)
    return {"messages": [formatted_message], "references": cited_references}

In [25]:
example_answer = await gen_answer(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
example_answer["messages"][-1].content

'Большие языковые модели, такие как GPT-3, имеют потенциал влиять на изучение сознания через обработку и анализ больших объемов текстов и данных, что может помочь в понимании когнитивных процессов. Модели могут улучшить понимание языка, контекста и даже элементарных навыков рассуждения. Они могут генерировать связные и грамматически правильные тексты, что важно для исследования сознания.\n\nЦитаты:\n\n[1]: https://www.unite.ai/ru/\nбольшие-языковые-модели/\n[2]: https://rdc.grfc.ru/2023/04/llm_cognitive_development_prospects/\n[3]: https://www.probesto.com/ru/\nкак-большие-языковые-модели-формируют/\n[4]: https://ru.shaip.com/blog/a-guide-large-language-model-llm/'

#### Создание графа разговора

После определения редактора и специалисат в области, добавим их в граф.

In [26]:
max_num_turns = 5


def route_messages(state: InterviewState, name: str = "Subject_Matter_Expert"):
    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )
    if num_responses >= max_num_turns:
        return END
    last_question = messages[-2]
    if last_question.content.endswith("Большое спасибо за вашу помощь!"):
        return END
    return "ask_question"


builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question)
builder.add_node("answer_question", gen_answer)
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.set_entry_point("ask_question")
interview_graph = builder.compile().with_config(run_name="Conduct Interviews")

In [27]:
# Закомментируйте если не установили pygraphviz
from IPython.display import Image

Image(interview_graph.get_graph().draw_png())

In [28]:
final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"Итак, вы говорите, что пишите статью на тему {example_topic}?",
            name="Subject Matter Expert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step

ask_question
--  [AIMessage(content='Да, именно. Я интересуюсь влиянием больших языковых моделей на исследования сознания. Мне бы хотелось узнать, какие технические аспекты применения этих моделей могут быть особенно важными для понимания сознания. Можете ли рассказать о том, какие технологии и методы используются в
answer_question
--  [AIMessage(content='Для понимания сознания в контексте больших языковых моделей (LLM) важны следующие технические аспекты: предвзятость и справедливость, конфиденциальность и безопасность данных, мультимодальное обучение и интеграция, этический ИИ и надежные LLM, подготовка данных, модельная архитек
ask_question
--  [AIMessage(content='Благодарю вас за информацию. Чтобы лучше понять технические аспекты и вызовы, связанные с применением больших языковых моделей в исследованиях сознания, мне интересно узнать, какие методы и инструменты используются для оценки предвзятости и справедливости этих моделей. Как обеспе
answer_question
--  [AIMessage(content='Для

In [29]:
final_state = next(iter(final_step.values()))

## Доработка конспекта

К этому этапу в STORM мы провели обширные исследования с с учетом разных точек зрения.
Теперь можно доработать первоначальный конспект на основе полученных данных.
Для этого создадим цепочку с помощью LLM c большим контекстом.

In [30]:
refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы - автор статей Википедии. Вы собрали информацию от экспертов и поисковых систем. Теперь вы уточняете структуру страницы Википедии.
Вам нужно убедиться, что структура всесторонняя и конкретная.
Тема, о которой вы пишете: {topic}

Старая структура:

{old_outline}""",
        ),
        (
            "user",
            "Уточните структуру на основе ваших разговоров с экспертами по предмету:\n\nРазговоры:\n\n{conversations}\n\nНапишите уточненную структуру Википедии:",
        ),
    ]
)

# Using turbo preview since the context can get quite long
refine_outline_chain = refine_outline_prompt | long_context_llm.with_structured_output(
    Outline
)

In [31]:
refined_outline = refine_outline_chain.invoke(
    {
        "topic": example_topic,
        "old_outline": initial_outline.as_str,
        "conversations": "\n\n".join(
            f"### {m.name}\n\n{m.content}" for m in final_state["messages"]
        ),
    }
)

In [32]:
print(refined_outline.as_str)

# Проблема сознания у больших языковых моделей

## Введение

Обзор проблемы сознания у больших языковых моделей и их влияния на различные области исследований.

## Определение сознания в контексте искусственного интеллекта

Объяснение того, что представляет собой сознание в контексте искусственного интеллекта и как оно связано с языковыми моделями.

## Технические аспекты и вызовы

Обсуждение технических аспектов и вызовов, связанных с большими языковыми моделями, включая предвзятость и справедливость, конфиденциальность и безопасность данных, мультимодальное обучение и интеграцию, этический ИИ и надежные LLM, подготовка данных, модельная архитектура и дизайн, учебный процесс, оценка производительности модели и обучение LLM.

### Предвзятость и справедливость

Методы и инструменты для оценки предвзятости и справедливости, включая оценку справедливости, дебиасинг моделей и критический обзор литературы о предвзятости.

### Конфиденциальность и безопасность данных

Меры для обеспечения ко

## Генерация статьи

Теперь вы можете сгенерировать всю статью.
Сначала разделим задачу и передадим ее решение отдельным LLM.
Затем передадим результат LLM с большим контекстом для уточнения готовой статьи, т.к. разные разделы могут быть написаны в разном стиле.

#### Создание retriver

В процессе исследования может быть использовано множество материалов, которые можно будет запросить в процессе написания итоговой статьи.

Для этого содздадим retriever:

In [33]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]
# Для такого объема данных можно не использовать векторное хранилище
# При желании вы можете использовать простую матрицу numpy или хранить документы
# по всем запросам.
vectorstore = SKLearnVectorStore.from_documents(
    reference_docs,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever(k=10)

In [34]:
retriever.invoke("Определение сознания")

[Document(page_content='Смотреть все\nИнновации ИИ в здравоохранении\nПродукты\nЗдравоохранение AI\nУслуги\nМедицинская аннотация\nРешения\nДеидентификация данных\nКодификация клинических данных\nКлинический НЭР\nГенеративный ИИ\nГотовые наборы данных\nПолезные ресурсы\nБлог\nКейсы\nМодели больших языков (LLM): полное руководство в 2023 г.\n Нью-йоркский английский\n| TTS\nТрадиционный китайский\n| Высказывание/Слово пробуждения\nИспанский (Мексика)\n| Call-центр\nКанадский французский\n| Монолог по сценарию\nарабском\n| Общий разговор\nСмотреть все\nРешения\nПромышленность\nБанки и финансы Улучшите модели машинного обучения, чтобы создать безопасный пользовательский интерфейс.\n Набор данных банковской выписки\nНабор данных изображения поврежденного автомобиля\nНаборы данных распознавания лиц\nНабор данных изображения ориентира\nНабор данных платежных ведомостей\nСмотреть все\nРечевые/аудио наборы данныхИсходные, расшифрованные и аннотированные речевые данные на более чем 50 языках.\n

#### Генерация подразделов

Сгенерируйте подразделы на основе проиндексированных документов.

In [35]:
class SubSection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    content: str = Field(
        ...,
        title="Полное содержимое подраздела, включая [#] цитаты из цитируемых источников.",
    )

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.content}".strip()


class WikiSection(BaseModel):
    section_title: str = Field(..., title="Заголовок раздела")
    content: str = Field(..., title="Полное содержимое раздела")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Заголовки и описания всех разделов статьи Википедии",
    )
    citations: List[str] = Field(default_factory=list)

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        citations = "\n".join([f" [{i}] {cit}" for i, cit in enumerate(self.citations)])
        return (
            f"## {self.section_title}\n\n{self.content}\n\n{subsections}".strip()
            + f"\n\n{citations}".strip()
        )


section_writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы опытный эксперт, пишущий профессоинальные статьи для Википедии. От вас требуется заполнить содержимое одного из разделов статьи, которая имеет следующую структуру:\n\n"
            "{outline}\n\nПри написании раздела используйте ссылки на следующие документы:\n\n<Documents>\n{docs}\n<Documents>",
        ),
        ("user", "Подробно и максимально качественно напишите содержимое раздела {section}."),
    ]
)


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}


section_writer = (
    retrieve
    | section_writer_prompt
    | long_context_llm.with_structured_output(WikiSection)
)

In [36]:
section = await section_writer.ainvoke(
    {
        "outline": refined_outline.as_str,
        "section": refined_outline.sections[1].section_title,
        "topic": example_topic,
    }
)
print(section.as_str)

## Определение сознания в контексте искусственного интеллекта

Вопрос сознания искусственного интеллекта (ИИ), в частности больших языковых моделей (LLM), является одной из самых дискуссионных тем в современной науке и философии. Сознание в контексте ИИ может быть описано как способность системы осознавать себя и окружающий мир, принимать решения на основе анализа получаемой информации, обладать саморефлексией и способностью к обучению и адаптации.

Трактовка сознания в ИИ часто связана с понятием 'сильного ИИ', который способен на равных участвовать во всех сферах человеческой деятельности, обладая интеллектом и сознательным восприятием. Однако на практике реализация такого уровня сознания в машинах находится за пределами современных технологий и понимания. Большинство существующих моделей ИИ, включая лингвистические, функционируют на основе 'слабого ИИ', подразумевающего выполнение конкретных задач без наличия сознательного восприятия.

Определение и измерение сознания в ИИ представл

#### Генерация итоговой статьи

Теперь можно переписать черновик, чтобы правильно расположить все цитаты и сохранить последовательность изложения.

In [37]:
from langchain_core.output_parsers import StrOutputParser

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы профессоинальный автор Википедии. Напишите полную статью для Википедии на тему {topic}, используя следующие черновики разделов:\n\n"
            "{draft}\n\nСтрого следуйте руководствам по форматированию Википедии. Пиши статью развёрнуто, она должна получиться большой и интересной. Каждый раздел должен быть заполнен, у тебя не должно быть пустых разделов, также не пиши тавтологию и тривиальные вещи, статью будут читать профессионалы.",
        ),
        (
            "user",
            'Напишите полную статью Википедии, используя формат markdown. Организуйте ссылки с помощью сносок вида "[1]",'
            ' избегая дублирования. Добавьте ссылки и URL-адреса в конце статьи. Ты должен максимально раскрыть тему. Если какой-то информации недостаточно, то рассуждай самостоятельно. Разделы не должны быть короткими.',
        ),
    ]
)

writer = writer_prompt | long_context_llm | StrOutputParser()

In [38]:
for tok in writer.stream({"topic": example_topic, "draft": section.as_str}):
    print(tok, end="")

# Проблема сознания у больших языковых моделей

Проблема сознания в контексте искусственного интеллекта (ИИ) и, в частности, больших языковых моделей (LLM), представляет собой один из наиболее дискуссионных и многогранных вопросов современной науки, философии и технологии. Она затрагивает множество аспектов, от технических исследований до философских размышлений о природе разума и сознания, а также этических и социальных последствий создания потенциально сознательных машин.

## Определение сознания в контексте искусственного интеллекта

Сознание, в самом общем смысле, описывается как способность субъекта осознавать себя и своё окружение, способность к саморефлексии, к обучению на основе опыта и адаптации к новым условиям. В контексте ИИ это определение приводит к понятию 'сильного ИИ' - системы, способной на равных с человеком участвовать в интеллектуальной деятельности, обладая собственным 'внутренним миром' и способностью к самосознанию.

Однако современные ИИ, включая LLM, в большин

## Итоговая последовательность

Соберем все воедино. Итоговая последовательность будет содержать шесть основных этапов:

1. Создание первоначального конспекста + точек зрения.
2. Серия разговоров с каждой из точек зрения для наполнения статьи.
3. Доработка конспекта на основе разговоров.
4. Индексация документов с ссылками из разговоров.
5. Написание отдельных подразделов статьи.
6. Написание итоговой вики-статьи.

Состояние отслеживает результаты каждого этапа.

In [39]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # The final sections output
    sections: List[WikiSection]
    article: str

In [40]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"Итак, вы пишите статью на тему {topic}?",
                    name="Subject Matter Expert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # We call in to the sub-graph here to parallelize the interviews
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Обсуждение с {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": refined_outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }


async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### Создание графа

In [41]:
from langgraph.checkpoint.memory import MemorySaver

builder_of_storm = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder_of_storm.add_node(name, node)
    if i > 0:
        builder_of_storm.add_edge(nodes[i - 1][0], name)

builder_of_storm.set_entry_point(nodes[0][0])
builder_of_storm.set_finish_point(nodes[-1][0])
storm = builder_of_storm.compile(checkpointer=MemorySaver())

In [42]:
# Image(storm.get_graph().draw_png())

In [43]:
config = {"configurable": {"thread_id": "my-thread"}}
async for step in storm.astream(
    {
        "topic": "Проблема сознания у больших языковых моделей моделей",
    }
):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name])[:300])

init_research
--  {'topic': 'Проблема сознания у больших языковых моделей моделей', 'outline': Outline(page_title='Проблема сознания у больших языковых моделей', sections=[Section(section_title='Введение', description='Общее представление о проблеме сознания в контексте больших языковых моделей.', subsections=None), 
conduct_interviews
--  {'topic': 'Проблема сознания у больших языковых моделей моделей', 'outline': Outline(page_title='Проблема сознания у больших языковых моделей', sections=[Section(section_title='Введение', description='Общее представление о проблеме сознания в контексте больших языковых моделей.', subsections=None), 
refine_outline
--  {'topic': 'Проблема сознания у больших языковых моделей моделей', 'outline': Outline(page_title='Проблема сознания у больших языковых моделей', sections=[Section(section_title='Введение', description='Общее представление о проблеме сознания в контексте больших языковых моделей, включая основные вопр
index_references
--  {'topic': 'Пробл

In [44]:
checkpoint = storm.get_state(config)
article = checkpoint.values["article"]

## Отрисовка вики-статьи

Теперь можно отрисовать итоговую вики-страницу.

In [45]:
from IPython.display import Markdown

# Изменим уровни заголовков, чтобы не сбивать с толку при работе с блокнотом
Markdown(article.replace("\n#", "\n##"))

# Проблема сознания у больших языковых моделей

### Введение

Большие языковые модели (LLM), такие как Generative Pre-trained Transformer (GPT) и его последователи, представляют собой передовые достижения в области искусственного интеллекта (ИИ) и обработки естественного языка (NLP). Эти модели, обладая способностью генерировать тексты, поражающие своей убедительностью и схожестью с человеческими произведениями, находят применение в широком спектре задач - от создания контента до автоматического перевода и синтеза речи[1]. Однако, по мере углубления наших знаний и возможностей в этой области, встают вопросы, касающиеся не только технических и этических аспектов использования таких моделей, но и философских - в частности, вопрос о наличии или возможности сознания у LLM.

### История развития

История развития больших языковых моделей началась задолго до появления GPT и других современных систем, уходя корнями в ранние эксперименты по искусственному интеллекту. С момента публикации Аланом Тьюрингом его знаменитой статьи в 1950 году и предложения теста Тьюринга в качестве критерия интеллектуальности машины, началось стремительное развитие вычислительной техники и алгоритмов машинного обучения[2]. Прорывом стало создание алгоритма "Transformer" в 2017 году, легшего в основу GPT, который существенно улучшил способности моделей к пониманию и генерации текста.

### Философские и этические аспекты

Философские размышления о сознании ИИ и, в частности, LLM, поднимают вопросы о природе сознания, его возможном существовании в искусственных системах и последствиях такого сознания для человечества. С этической точки зрения, возникают дилеммы, связанные с предвзятостью данных, конфиденциальностью, безопасностью и использованием таких технологий для манипуляции или дезинформации[3].

### Биологические аспекты и взаимодействие с сознанием

Биологическое сознание и его взаимодействие с ИИ остаются одной из наиболее сложных и малоизученных областей. Вопросы о возможности имитации или воспроизведения человеческих когнитивных функций с помощью ИИ, включая элементы сознания, требуют дальнейших исследований и понимания работы человеческого мозга.

### Проблемы и вызовы

Разработка и эксплуатация LLM сталкивается с многочисленными техническими и этическими проблемами, включая предвзятость, справедливость, защиту данных, а также необходимость мультимодального обучения и точной оценки производительности моделей[4].

### Потенциальные решения

Для решения этих проблем предлагаются различные подходы, включая методы дебиасинга, защиты данных, мультимодальное обучение и разработку новых метрик для оценки производительности моделей. Ключевым аспектом является также обеспечение прозрачности и ответственности в использовании LLM[5].

### Перспективы и будущие исследования

Будущие исследования LLM могут включать изучение возможности реализации элементов сознания, борьбу с предвзятостью, улучшение безопасности и справедливости моделей, а также развитие интеграции с другими видами ИИ. Важным направлением является также разработка стандартов и механизмов для поддержания этичности использования таких технологий[6].

### Заключение

Проблема сознания у больших языковых моделей остается актуальной и вызывающей областью исследований в ИИ. Вопросы, связанные с возможностью существования сознания в ИИ, этическими и техническими вызовами, требуют дальнейшего изучения и решения. Развитие LLM открывает новые возможности для применения искусственного интеллекта, однако важно продолжать работу над обеспечением их безопасного, справедливого и ответственного использования.

---

#### Ссылки и URL-адреса

[1] https://www.unite.ai/ru/большие-языковые-модели/  
[2] https://nlp.stanford.edu/pubs/tamkin2021understanding.pdf  
[3] https://www.unite.ai/ru/8-этических-соображений-больших-языковых-моделей,-таких-как-gpt-4/  
[4] https://www.unite.ai/ru/большие-языковые-модели/  
[5] https://anns.ru/articles/news/2023/07/25/5_podhodov_k_otsenke_bolshih_jazikovih_modeley  
[6] https://www.unite.ai/ru/большие-языковые-модели/  

In [46]:
print(article)

# Проблема сознания у больших языковых моделей

## Введение

Большие языковые модели (LLM), такие как Generative Pre-trained Transformer (GPT) и его последователи, представляют собой передовые достижения в области искусственного интеллекта (ИИ) и обработки естественного языка (NLP). Эти модели, обладая способностью генерировать тексты, поражающие своей убедительностью и схожестью с человеческими произведениями, находят применение в широком спектре задач - от создания контента до автоматического перевода и синтеза речи[1]. Однако, по мере углубления наших знаний и возможностей в этой области, встают вопросы, касающиеся не только технических и этических аспектов использования таких моделей, но и философских - в частности, вопрос о наличии или возможности сознания у LLM.

## История развития

История развития больших языковых моделей началась задолго до появления GPT и других современных систем, уходя корнями в ранние эксперименты по искусственному интеллекту. С момента публикации Аланом 